In [8]:
import torch
import numpy as np
import pandas as pd
import torch.utils
from torch import nn

In [9]:
#Loading house prices training and test dataset
hptrain = pd.read_csv("Datasets/hptrain.csv",index_col=0)
hptest = pd.read_csv("Datasets/hptest.csv",index_col=0)

In [10]:
#Transforming data into tensor data format from pandas' dataframe
X_ols_train = torch.tensor(hptrain[['LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'TotalBsmtSF']].values,requires_grad=True,dtype=torch.float32)
X_ols_train = nn.functional.normalize(X_ols_train)
Y_ols_train = torch.tensor(hptrain[['SalePrice']].values,requires_grad=True,dtype=torch.float32)


#Splitting data into iterable batches
olstensor = torch.utils.data.TensorDataset(X_ols_train,Y_ols_train)
olsdataloader = torch.utils.data.DataLoader(olstensor, batch_size=20,shuffle=True)

In [11]:
#Specifying the paramenters of our model
olsnet = nn.Sequential(nn.Linear(X_ols_train.size()[1],Y_ols_train.size()[1]))
olsnet[0].weight.data.normal_(0, 0.1)
olsnet[0].bias.data.normal_(0, 0.1)
olsloss = nn.MSELoss()
olstrainer = torch.optim.SGD(olsnet.parameters(),lr=1)

In [12]:
num_epochs = 10
olsnet.train()
for epoch in range(num_epochs):
    print('-------------------')
    print(f'{epoch+1}th epoch')
    print('===================')
    olstrainer.zero_grad()
    X, Y = next(iter(olsdataloader))
    output = olsnet(X)

    l = olsloss(output,Y)
    print(f'Loss ols:{l}')


    l.backward(retain_graph=True)

    olstrainer.step()
    print('Coefficients: ',olsnet[0].weight.data)


-------------------
1th epoch
Loss ols:38776840192.0
Grad bef:None
Grad aft:None
Coefficients:  tensor([[3.5455e+05, 2.3418e+02, 2.1216e+02, 2.6712e+03, 4.6565e+04]])
-------------------
2th epoch
Loss ols:269102710784.0
Grad bef:None
Grad aft:None
Coefficients:  tensor([[-6.5992e+05, -4.3809e+02, -3.9943e+02, -7.9720e+03, -5.8545e+04]])
-------------------
3th epoch
Loss ols:2191683616768.0
Grad bef:None
Grad aft:None
Coefficients:  tensor([[2.2619e+06, 1.5828e+03, 1.5199e+03, 1.4874e+04, 3.3997e+05]])
-------------------
4th epoch
Loss ols:19131923431424.0
Grad bef:None
Grad aft:None
Coefficients:  tensor([[-6.4008e+06, -5.1323e+03, -4.4069e+03, -1.0968e+05, -7.4989e+05]])
-------------------
5th epoch
Loss ols:170607576612864.0
Grad bef:None
Grad aft:None
Coefficients:  tensor([[1.9488e+07, 1.1642e+04, 9.7319e+03, 1.6820e+05, 2.1959e+06]])
-------------------
6th epoch
Loss ols:1523008556498944.0
Grad bef:None
Grad aft:None
Coefficients:  tensor([[-5.7350e+07, -4.7322e+04, -4.7366e+

/Users/bartoszgawin/lib/python3.10/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:482.)
  return self._grad


In [ ]:
#Loading in complex MNIST train and test datasets